In [1]:
import json

In [2]:
from pyspark import SparkConf,SparkContext
spark_conf = SparkConf().setMaster("local").setAppName("ReadHbase")
spark_context = SparkContext(conf = spark_conf)
# from pyspark.sql import SparkSession
# spark_session = SparkSession(spark_context)

In [28]:
hbase_host = "192.168.1.150"
hbase_table = "TEST_RPT"
hbase_conf = {"hbase.zookeeper.quorum": hbase_host,
              "hbase.mapreduce.inputtable": hbase_table,
              "hbase.mapreduce.scan.row.start": '18589801',
              "hbase.mapreduce.scan.row.stop" : '18589802',
              "hbase.mapreduce.scan.columns"  : "info:cont",
              # "hbase.mapreduce.scan.columns"  : "info:cont info:bd info:sex",
             }
              
keyConv = "org.apache.spark.examples.pythonconverters.ImmutableBytesWritableToStringConverter"
valueConv = "org.apache.spark.examples.pythonconverters.HBaseResultToStringConverter"

In [29]:
hbase_rdd = spark_context.newAPIHadoopRDD("org.apache.hadoop.hbase.mapreduce.TableInputFormat",
                                          "org.apache.hadoop.hbase.io.ImmutableBytesWritable",
                                          "org.apache.hadoop.hbase.client.Result",
                                          keyConverter=keyConv,
                                          valueConverter=valueConv,
                                          conf=hbase_conf
                                         )

In [30]:
count = hbase_rdd.count()
count

1

In [31]:
hbase_rdd.cache()
output = hbase_rdd.collect()
for (k, v) in output[:1]:
    print(v)
    # raw = v.encode('raw_unicode_escape').decode('utf-8')
    # print(raw[:250])
    # raw_data = json.loads(v)["value" ]
    # print(raw_data)
    # print(raw_data.encode('raw_unicode_escape').decode('utf-8'))

{"qualifier" : "cont", "timestamp" : "1574216805668", "columnFamily" : "info", "row" : "18589801", "type" : "Put", "value" : "{\"checkItems\":[{\"checkItemName\":\"\\xE4\\xB8\\x80\\xE8\\x88\\xAC\\xE6\\xA3\\x80\\xE6\\x9F\\xA5\",\"checkResults\":[{\"appendInfo\":\"\",\"canExplain\":0,\"checkIndexName\":\"\\xE6\\x94\\xB6\\xE7\\xBC\\xA9\\xE5\\x8E\\x8B\",\"highValueRef\":\"\",\"isAbandon\":null,\"isCalc\":null,\"lowValueRef\":\"\",\"resultFlagId\":1,\"resultTypeId\":null,\"resultValue\":\"127\",\"showIndex\":null,\"textRef\":\"\",\"unit\":\"mmHg\"},{\"appendInfo\":\"\",\"canExplain\":0,\"checkIndexName\":\"\\xE8\\x88\\x92\\xE5\\xBC\\xA0\\xE5\\x8E\\x8B\",\"highValueRef\":\"\",\"isAbandon\":null,\"isCalc\":null,\"lowValueRef\":\"\",\"resultFlagId\":1,\"resultTypeId\":null,\"resultValue\":\"73\",\"showIndex\":null,\"textRef\":\"\",\"unit\":\"mmHg\"},{\"appendInfo\":\"\",\"canExplain\":1,\"checkIndexName\":\"\\xE8\\xBA\\xAB\\xE9\\xAB\\x98\",\"highValueRef\":\"\",\"isAbandon\":null,\"isCalc\":nu

In [32]:
"aa\xE4\xB8\x80\xE8\x88\xAC\xE6\xA3\x80\xE6\x9F\xA5aa".encode('raw_unicode_escape').decode('utf-8')

'aa一般检查aa'